In [161]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [162]:
import wntr
from pathlib import Path
import wntr.network.controls as controls
from dreaminsg_integrated_model.src.network_sim_models.interdependencies import *
from dreaminsg_integrated_model.src.network_recovery import *
import pandapower as pp

import math
import pandapower.plotting as pandaplot

In [163]:
network_dir= 'micropolis'

MAIN_DIR = Path('../..')
water_file = MAIN_DIR/'dreaminsg_integrated_model/data/networks/{0}/water/water.inp'.format(network_dir)

step_size = 3600
try:
    wn = wntr.network.WaterNetworkModel(water_file)
    wn.options.time.duration = 3600*24
    wn.options.time.report_timestep = step_size
    wn.options.time.hydraulic_timestep = step_size
    wn.options.hydraulic.demand_model = 'DDA'
    
    print("Water network successfully loaded.")
except FileNotFoundError:
    print("Error: The water network file does not exist. No such file or directory: ", water_file)

Water network successfully loaded.


C:\Users\srijith\anaconda3\envs\redcar\lib\site-packages\wntr\epanet\io.py:2173: UserWarning: Not all curves were used in "..\..\dreaminsg_integrated_model\data\networks\micropolis\water\water.inp"; added with type None, units conversion left to user
  warnings.warn('Not all curves were used in "{}"; added with type None, units conversion left to user'.format(self.wn.name))


In [164]:
[wn.get_link(pump).status for pump in wn.pump_name_list]

[<LinkStatus.Opened: 1>,
 <LinkStatus.Opened: 1>,
 <LinkStatus.Opened: 1>,
 <LinkStatus.Opened: 1>,
 <LinkStatus.Opened: 1>,
 <LinkStatus.Closed: 0>,
 <LinkStatus.Closed: 0>,
 <LinkStatus.Opened: 1>]

In [165]:
# ax = wntr.graphics.plot_network(wn, node_attribute='base_demand',
# node_colorbar_label='Base demand', node_size = 0.5, link_width = 0.5)

In [166]:
# #wn.get_link('W_PMA2000').status = 0
# wn.get_link('W_PMA1007').status = 0
# wn.get_link('W_PMA2000')

In [167]:
component = 'W_PMA505'
wn = wntr.morph.split_pipe(wn,component,f"{component}_B",f"{component}_leak_node",)

leak_node = wn.get_node(f"{component}_leak_node")
leak_node.remove_leak(wn)
leak_node.add_leak(wn,area=0.75*(math.pi*(wn.get_link(f"{component}_B").diameter)** 2)/4,
    start_time=6000,
    end_time=18000,
)

link_close_event(wn, component, 18000, "repairing")
link_open_event(wn, component, 30000, "repairing")

In [168]:
print("Simulation time: ", wn.options.time.duration,"; Hydraulic time step: ", wn.options.time.hydraulic_timestep, "; Report time step: ", wn.options.time.report_timestep)
wn_sim = wntr.sim.WNTRSimulator(wn)
wn_results = wn_sim.run_sim(convergence_error=True, solver_options={"MAXITER": 10000})

print('failed pipe flowrate: ', wn_results.link['flowrate'][component].values)
print('failed pipe leak: ', wn_results.node['leak_demand'][f'{component}_leak_node'].values)
print('tank demand: ', wn_results.node['head']['W_T1'].values)
print('pump flow rate: ', wn_results.link['flowrate'][wn.pump_name_list].values)

Simulation time:  86400.0 ; Hydraulic time step:  3600.0 ; Report time step:  3600


C:\Users\srijith\anaconda3\envs\redcar\lib\site-packages\scipy\optimize\minpack.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',


failed pipe flowrate:  [-0.003 -0.003  0.35   0.316  0.031  0.     0.     0.     0.    -0.003 -0.003 -0.003 -0.003 -0.003 -0.004 -0.004 -0.004 -0.004 -0.004 -0.003 -0.003 -0.003 -0.003 -0.003 -0.003]
failed pipe leak:  [0.    0.    0.354 0.32  0.035 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.   ]
tank demand:  [352.044 352.684 350.922 344.585 344.424 344.424 344.587 344.58  344.512 344.424 344.424 344.424 344.424 344.424 344.424 344.424 344.424 344.424 344.424 344.424 344.424 344.647 345.1   345.663 346.326]
pump flow rate:  [[0.001 0.001 0.001 0.001 0.068 0.    0.    0.064]
 [0.001 0.001 0.001 0.001 0.068 0.    0.    0.064]
 [0.002 0.002 0.002 0.002 0.071 0.    0.    0.064]
 [0.003 0.003 0.003 0.003 0.074 0.    0.    0.064]
 [0.006 0.006 0.006 0.006 0.087 0.    0.    0.064]
 [0.002 0.002 0.002 0.002 0.072 0.    0.    0.064]
 [0.002 0.002 0.002 0.002 0.073 0.    0.    0.064]
 [0.002 0.002 0.002 0.002 0.073 0.    0

In [169]:
wn_results.node['demand'].to_csv('results.csv')

In [170]:
pd.Series(water_consump_tracker).plot()

NameError: name 'water_consump_tracker' is not defined

In [ ]:
t = 86400
node_results = wn_results.node['demand'].iloc[-1]

water_supplied = sum([node_results[junc] for junc in wn.junction_name_list])

base_demands_at_t = []

for junc in wn.junction_name_list:
    base_demand = wn.get_node(junc).base_demand
    
    if base_demand != 0:
        demand = node_results[junc]
        pattern = wn.get_node(junc).demand_timeseries_list[0].pattern.multipliers
        #print(pattern)
        pattern_size = len(pattern)
        pattern_interval = 24/pattern_size
        
        pattern_index = math.ceil(((t/3600)%86400) / pattern_interval)
        multiplier = pattern[pattern_index-1]
        base_demands_at_t.append(multiplier*base_demand)
        

water_resmetric = water_supplied/sum(base_demands_at_t)
print(water_resmetric)

1.0


In [171]:
wn.pump_name_list

['W_WP1', 'W_WP2', 'W_WP3', 'W_WP4', 'W_WP5', 'W_WP6', 'W_WP7', 'W_WP8']